# Question 6

In [ ]:
import numpy as np
from tqdm import tqdm # Nice Progress bar for loops
import matplotlib.pyplot as plt
from Rankine_solver import define_empty_variables, solve_r_rankine_cycle
from Rankine_solver import display_tables, plot_Ts_cycle

In [ ]:
# === CONSTANTS ===

''' T2_Range has to be changed to -10C to 35C when Romeo fixes code'''

T2_range = np.linspace(-10, 31, 100)  # 100 points between -10°C and 35°C 
m_dot_max = 0.16  # [kg/s], given from earlier question (minimum of hot and cold limits)

def get_W_c(T2_C):
    vars = define_empty_variables()
    vars['2']['T'] = T2_C  # C
    vars['4']['T'] = 35 # C
    vars['Qh_dot']= 8.18 #kW
    solved = solve_r_rankine_cycle(vars)
    m_dot = solved['m_dot']
    Wc_dot = solved['Wc_dot']
    

    return(Wc_dot,m_dot)

# === Main Calculation ===
T2_list = []
W_c_list = []
m_dot_list = []
solid_line = []

for T2_C in T2_range:
    Wc_dot,m_dot = get_W_c(T2_C)
    
    #Store
    T2_list.append(T2_C)  # back to Celsius
    W_c_list.append(float(Wc_dot))    # kW
    m_dot_list.append(float(m_dot))
    solid_line.append(float(m_dot) <= m_dot_max)


# === Plotting ===
plt.figure(figsize=(10,6))

T2_list = np.array(T2_list)
W_c_list = np.array(W_c_list)
solid_line = np.array(solid_line)

# Solid line where m_dot <= m_dot_max
plt.plot(T2_list[solid_line], W_c_list[solid_line], 'b-', label='ṁ ≤ ṁ_max')

# Dashed line where m_dot > m_dot_max
plt.plot(T2_list[~solid_line], W_c_list[~solid_line], 'b--', label='ṁ > ṁ_max')

# Mark the transition point
if np.any(~solid_line):
    idx_transition = np.argmax(~solid_line)
    plt.plot(T2_list[idx_transition], W_c_list[idx_transition], 'ro', label='ṁ = ṁ_max')

plt.xlabel('T₂ (°C)')
plt.ylabel('Compressor Input Power Ẇc (kW)')
plt.title('Compressor Input Power vs T₂')
plt.grid(True)
plt.legend()
plt.show()

# === Discussion ===
min_idx = np.argmin(W_c_list)
print(f"Minimal power consumption: {W_c_list[min_idx]:.2f} kW at T₂ = {T2_list[min_idx]:.2f} °C")

if np.isclose(T2_list[min_idx], T4_C, atol=0.5):
    print("T₂ approaches T₄: compression work theoretically approaches zero.")
else:
    print("T₂ is significantly lower than T₄.")
